# odc-stac tutorial - solution

> **NOTE** This is the solution, which contains executable code. 
> Use this notebook to confirm what the correct code is if you get stuck.

In this tutorial we will use Python libraries to find and load land cover data from the freely available [Impact Observatory Annual Land Use Land Cover](https://planetarycomputer.microsoft.com/dataset/io-lulc-annual-v02) product.
After loading the data, we will export it as Cloud Optimised GeoTIFFs.
This will allow you to further view or work with the data in GIS software and other tools.

During the tutorial, we will:

* Specify our search in terms of:

  * what (data provider and satellite)
  * where (area of interest)
  * when (date range)
* Use `pystac-client` to connect to a Spatio-Temporal Asset Catalog (STAC) 
  endpoint and search for data matching our what, where, and when
* Use `odc-stac` to load the matching data into memory
* Visualise and export the data

## Requirements
Please keep a copy of the [tutorial instructions](https://opendatacube.readthedocs.io/en/latest/tutorials/odc-stac.html) open.

Follow the instructions for each labelled section. 
When you have finished entering code into a cell, press `Shift+Enter` on your keyboard to run the cell.

## Python imports

In this section, you will import five packages that the notebook needs to run:

* `geopandas` for loading an area of interest from a 
* `odc.geo` for exporting loaded data
* `odc.stac` for loading data
* `planetary_computer` to provide authentication when accessing data
* `pystac-client` for querying catalogs of data

In [ ]:
import geopandas as gpd
from odc.geo.xr import write_cog
from odc.stac import load
import planetary_computer
from pystac_client import Client

## Set up query parameters

In this section of the tutorial, you will specify:

* The area you want to load data for
* The date range you want to load data for
* The data source you want to load from

### Area of interest

We specify the area of interest using the `aoi.geojson` file, which can be loaded with `geopandas`.

The area of interest is the southern part of Isla Isabela, one of the islands in the Galapagos.

![A satellite image of Isla Isabela, with the area of interest shown as a yellow bounding box.](images/aoi.png "Isla Isabela area of interest")

In [ ]:
aoi = gpd.read_file("aoi.geojson")

aoi_geometry = aoi.iloc[0].geometry

### Date range

We specify the date range by providing a start and end date for our query, then storing these in a tuple.

In [ ]:
start_date = "2017-01-01"
end_date = "2023-01-01"

date_query = (start_date, end_date)

### Catalogs, collections, and items

When working with STAC, we need to specify:

* The STAC catalog to search as `"https://planetarycomputer.microsoft.com/api/stac/v1/"`
* The collection to search as `["io-lulc-annual-v02"]`
* The assets to load as `["data"]`

In [ ]:
catalog_query = "https://planetarycomputer.microsoft.com/api/stac/v1/"
collections_query = ["io-lulc-annual-v02"]
assets_query = ["data"]

## Connect to catalog and find items

This section returns STAC items that match the query, each of which points to a Cloud Optimised GeoTIFF (COG) that contains the relevant data

In [ ]:
# Connect to the catalog
stac_client = Client.open(
    catalog_query, 
    modifier=planetary_computer.sign_inplace,
)

### Search for items

This section searches for items that match the query using `pystac-client`.
The search function has a few arguments:

- `collections`: a list of collections to search
- `intersects`: an area of interest to search. Any item intersecting the geometry will be returned.
- `datetime`: a date range to search

This step returns STAC items that match the query, each of which points to a file that contains the relevant data.

In [ ]:
# Search for items matching the query
items = stac_client.search(
    collections=collections_query,
    intersects=aoi_geometry,
    datetime=date_query,
).item_collection()

print(f"Found {len(items)} items")

Once returned, it is possible to view relevant metadata for all discovered items.

In [ ]:
items

## Load items with odc-stac

After producing a list of items to load, we can use the `load` function from `odc-stac` to read the requested assets from the items and return them as xarrays. 
The load function has a few arguments:

- `items`: a list of STAC items to load
- `bands`: a list of assets to load for each item
- `geopolygon`: the area of interest to load the data for
- `crs`: the coordinate reference system to use. `utm` will provide the most appropriate UTM projection
- `resolution`: the resolution to load the items at, in the same units as the `crs`. If using `utm` the unit is metres


In [ ]:
ds = load(
    items=items,
    bands=assets_query,
    geopolygon=aoi_geometry,
    crs="utm",
    resolution=30
)

ds

## Visualise loaded data
To confirm that we have loaded the requested data, it is useful to visualise it. 
We can use `xarray`'s built-in plotting functionality to make a basic plot.

It is possible to further customise the plot, as demonstrated in the Microsoft Planetary Computer [example notebook](https://planetarycomputer.microsoft.com/dataset/io-lulc-annual-v02#Example-Notebook).

In [ ]:
# Plot a time series of the loaded data, with three images per row
ds.data.plot.imshow(col="time", col_wrap=3)

## Export loaded data

You may wish to use your loaded data in other applications, such as a GIS program, or in other Python workflows. 
As such, it is valuable to export each image as a Cloud Optimised GeoTIFF. 
The `odc-geo` library provides the `write_cog` function to generate and write these files from an `xarray`.

In [ ]:
years = ds.time.dt.strftime("%Y").values

for timestep in range(len(ds.time)):

    ds_single_year = ds.data.isel(time=timestep)
    
    write_cog(
        ds_single_year,
        f"LULC_{years[timestep]}.tif", 
        overwrite=True,
    )

## Tutorial complete!

Congratulations, you've used `pystac-client` to search for data in a public STAC catalog and `odc-stac` to load the data into an `xarray`.

Make sure you download the Cloud Optimised GeoTIFF files you created in the last step before exiting the tutorial.
You can do this by right-clicking each file in the file browser and selecting "Download".
Note that these files will not be accessible once you close this tutorial environment.